# Glavna koda
V tem delu je zapisan prvi del kode, kjer bomo testirali grafe za določene velikosti.
Koda je ravno tako opisana v [poročilu](https://github.com/mpracek/Graffiti-conjecture-194/blob/master/Porocilo/dolgo_porocilo.pdf)

In [42]:
import random
import operator
import math

def nasi_grafi(stevilo_vozlisc):
    """
    Funkcija, ki nam vrne enostavne povezane grafe na dolocenem stevilu vozlisc.
    """
    grafi = list(graphs.nauty_geng(str(stevilo_vozlisc)+" -c"))
    return grafi

def neodvisnostno_stevilo(G):
    """
    Vrne neodvistno število grafa G
    Za pomoc uporabimo independet_set() iz modula neusmerjenih grafov.
    """
    neodvisno = G.independent_set()
    dolzina = len(neodvisno)
    return dolzina


def naredi_podgraf(G, seznam):
    """
    Funkcija, ki nam za dan seznam vozlisc vrne podgraf, definiran na le teh
    Seznam je tukaj nabor vozlisc, ki nas zanimajo.
    """
    H = Graph(G, immutable=True)
    sosedi = []
    for vozlisce in H:
        nabor_sosedov = ()
        if vozlisce in seznam:
            for element in H[vozlisce]:
                if element in seznam:
                    a = list(nabor_sosedov)
                    a.append([vozlisce, element])
                    tuple(a)
        sosedi.append(nabor_sosedov)
    nov_graf = H.subgraph(vertices = seznam, edges = sosedi)
    return nov_graf


def lokalna_neodvisnost(G, vozlisce):
    """
    Vrne lokalno neodvistnost grafa G v vozliscu v.
    Lokalna neodvistnost je neodvistnost podgrafa, ki ga določajo sosedi vozlišča v,
    za nek v iz množice vozlišč.
    """
    mnozica = G[vozlisce]
    novGraf = naredi_podgraf(G, mnozica)
    lokalna = neodvisnostno_stevilo(novGraf)
    return lokalna

def povprecna(G):
    """
    Vrne povprečno lokalno neodvisnost grafa G
    """
    povprecje =  0
    for vozlisce in G:
        povprecje += lokalna_neodvisnost(G, vozlisce)
    povprecna_vrednost  = povprecje/ len(G)
    return povprecna_vrednost

def preverjanje_za_en_graf(G):
    """
    Preveri, ali za en graf velja, če je ta graf izjema.
    """
    if neodvisnostno_stevilo(G) <= 1 + povprecna(G):
            if G.hamiltonian_path() == None:
                graf = str(G)
                return "Ovrgli smo domnevo in izjema je" + 	graf
    else:
        return "Izjeme nismo ovrgli"

def preverjanje(stevilo_vozlisc):
    """
    Preveri veljavnost konjekture za vse grafe na določenem številu vozlišč.
    """
    for G in nasi_grafi(stevilo_vozlisc):
        if neodvisnostno_stevilo(G) <= 1 + povprecna(G):
            if G.hamiltonian_path() == None:
                graf = str(G)
                return "Ovrgli smo domnevo in izjema je" + 	graf
    return "Izjeme nismo ovrgli"


In [43]:
prvi_test = preverjanje(4)


# Genetski algoritem
Od tu naprej bodo vse funkcije bile uporabljene za definicijo [genetskega algoritma](https://en.wikipedia.org/wiki/Genetic_algorithm).

In [2]:
def poisson(t = 1, lambd = 1/2):
    """
    S to funkcijo določimo začetno stevilo vozlisc
    """
    stevilo_vozlisc = 0
    racunalo = 0
    while racunalo < t:
        stevilo_vozlisc += 1
        racunalo += random.expovariate(lambd)
    return stevilo_vozlisc


In [85]:
def zacetna_populacija(maksimalna):
    """
    Ta funkcija nam da grafe, na katerih bo opravljen prvi test.
    Maksimalna nam pove, koliko je največja (maksimalna) velikost vsake generacije.
    """
    stevilo_vozlisc = poisson(t = 5, lambd = 1/2)
    populacija = []
    stevec = 0
    while stevec < maksimalna:
        graf = graphs.RandomGNP(stevilo_vozlisc, random.uniform(0, 1))
        if graf.is_connected():
            populacija.append(graf)
            stevec += 1
    return populacija


In [48]:
def razporedi(maksimalna=None):
    """
    Izracuna kriterij, po katerem vse elemente razporedimo in razporedi elemente populacije.
    Navadno za populacijo uporabimo zacetna_populacijo(maksimalna).
    """
    slovar = dict()
    populacija = zacetna_populacija(maksimalna)
    for element in populacija:
        slovar[element] = neodvistnostno_stevilo(element)
    razporejena_populacija = sorted(slovar.items(), key = operator.itemgetter(1))
    nasa_populacija = []
    for osebek in razporejena_populacija:
        nasa_populacija.append(populacija[osebek[0]])
    return nasa_populacija


In [75]:
def testna_populacija(maksimalna):
    """
    Za zacetni parameter izberemo zacetna_populacija(), ki nam bo vrnila
    vse grafe zacetne velikosti.
    Najprej moramo določiti, kolikšen del začetne populacije bomo testirali
    V genetiki velja, da ostanejo zgolj najboljsi, zato bomo vzeli le najboljse,
    torej tiste, z minimalnim neodvistnostnim številom.
    """
    populacija = razporedi()
    if len(populacija) > maksimalna:
        testna_populacija = populacija[:maksimalna]
    else:
        testna_populacija = populacija
    return testna_populacija


In [91]:
def doda_povezavo(graf):
    """
    Spremeni graf tako, da mu doda povezavo
    """
    G = Graph(graf)
    if G.vertices() == []:
        return G
    else:
        vozlisca = graf.vertices()
        vozlisce1 = G.random_vertex()
        vozlisce2 = G.random_vertex()
        if vozlisce1 != vozlisce2:
            G.add_edge(vozlisce1, vozlisce2)
        return G


In [78]:
def odstrani_povezavo(graf):
    """
    Odstrani nakljucno povezavo iz grafa;
    Pazimo, da mora graf, ki ga dobimo, biti povezan.
    """
    kopija = Graph(graf, immutable=True)
    G = Graph(kopija)
    if G.vertices() == []:
        return G
    else:
        vozlisce1 = G.random_vertex()
        vozlisce2 = G.random_vertex()
        if vozlisce1 != vozlisce2:
            G.delete_edge(vozlisce1, vozlisce2)
            if G.is_connected():
                return G
            else:
                G.add_edge(vozlisce1, vozlisce2)


In [68]:
def mutacija_vozlisce(graf):
    """
    Doda vozlisce in mu nato doda nekaj povezav nanj.
    """
    graf = Graph(graf)
    dolzina = len(graf.vertices())
    graf.add_vertex()
    stevilo = randint(0,dolzina)
    vozlisca = graf.vertices()
    dodaj_vozlisca = random.sample(vozlisca, stevilo)
    for i in range(stevilo):
        graf.add_edge(novo, dodaj_vozlisca[i])
    return graf


In [87]:
def odstrani_vozlisce(graf):
    """
    Funkcija iz grafa odstrani vozlišče in vse povezave nanj.
    """
    graf = Graph(graf)
    if graf.vertices() == []:
        return graf
    else:
        vozlisce1 = graf.random_vertex()
        graf.delete_vertex(vozlisce1)
        if graf.is_connected():
            return graf


In [10]:
def mutacija(graf):
    """
    Na grafu lahko odstranimo, dodamo povezavo, dodamo, odstranimo vozlisce in vsako kombinacijo le teh.
    """
    p = random.uniform(0,1)
    if p <= 1/15:
        odstrani_vozlisce(graf)
    elif p > 1/15 and p <= 2/15:
         mutacija_vozlisce(graf)
    elif p > 2/15 and p <= 3/15:
        odstrani_povezavo(graf)
    elif p > 3/15 and p <= 4/15:
        doda_povezavo(graf)
    elif p > 4/15 and p <= 5/15:
        sprememba = odstrani_vozlisce(graf)
        mutacija_vozlisce(sprememba)
    elif p > 5/15 and p <= 6/15:
        sprememba = odstrani_vozlisce(graf)
        odstrani_povezavo(sprememba)
    elif p > 6/15 and p <= 7/15:
        sprememba = odstrani_vozlisce(graf)
        doda_povezavo(sprememba)
    elif p > 7/15 and p <= 8/15:
        sprememba = mutacija_vozlisce(graf)
        odstrani_povezavo(sprememba)
    elif p > 8/15 and p <= 9/15:
        sprememba = mutacija_vozlisce(graf)
        doda_povezavo(sprememba)
    elif p > 9/15 and p <= 10/15:
        sprememba = odstrani_povezavo(graf)
        doda_povezavo(sprememba)
    elif p > 10/15 and p <= 11/15:
        sprememba = mutacija_vozlisce(graf)
        sprememba2 = doda_povezavo(sprememba)
        odstrani_povezavo(sprememba2)
    elif p > 11/15 and p <= 12/15:
        sprememba = mutacija_vozlisce(graf)
        sprememba2 = doda_povezavo(sprememba)
        odstrani_povezavo(sprememba2)
    elif p > 12/15 and p <= 13/15:
        sprememba = mutacija_vozlisce(graf)
        sprememba2 = odstrani_vozlisce(sprememba)
        odstrani_povezavo(sprememba2)
    elif p > 13/15 and p <= 14/15:
        sprememba = mutacija_vozlisce(graf)
        sprememba2 = doda_povezavo(sprememba)
        odstrani_vozlisce(sprememba2)
    elif p > 14/15 and p <= 15/15:
        sprememba = mutacija_vozlisce(graf)
        sprememba2 = doda_povezavo(sprememba)
        sprememba3 = odstrani_povezavo(sprememba2)
        odstrani_vozlisce(sprememba3)


In [11]:
def nova_populacija(populacija = None, verjetnost = 0.05):
    """
    Vsak graf iz populacije z neko verjetnostjo spremenimo.
    Za zacetku za populacijo uporabimo testna_populacija(maksimalna)
    Ti spremenjeni grafi nam dajo novo populacijo.
    """
    naslednja_generacija = []
    for i in range(len(populacija)):
        q = random.uniform(0,1)
        r = random.uniform(0,1)
        prob = q * r
        if verjetnost < prob:
            naslednja_generacija[i] = mutacija(populacija[i])
        else:
            naslednja_generacija[i] = populacija[i]
    return naslednja_generacija


In [12]:
def crossover(n, stars1, stars2):
    """
    V nasem primeru je n stevilo vozlisc, na katerih se grafa krizata.
    Nove povezave nam povedo, koliko in katere povezav bo imel graf.
    """
    while True:
        graf1 = stars1.random_subgraph(0.5)
        graf2 = stars2.random_subgraph(0.5)
        if len(graf1.vertices()) + len(graf2.vertices()) == n and len(graf1.vertices()) >= 1 and len(graf1.vertices()) < n and graf1.is_connected() and graf2.is_connected():
            graf1.relabel()
            graf2.relabel()
            naslednik = graf1.disjoint_union(graf2)
            nove_povezave = poisson(lambd = log(n/2))
            for k in range(nove_povezave):
                prvi = graf1.random_vertex()
                drugi = graf2.random_vertex()
                potomec.add_edge((0, prvi), (1, drugi))
            naslednik.relabel()
            break
    return potomec


In [13]:
def potomci(populacija = None, verjetnost = 0.05):
    """
    Navadno za populacijo uporabimo nova_populacija(populacija = testna_populacija(maksimalna))
    Funkcija nam vrne prejšnjo populacijo in premešano populacijo, združeno v eno.
    """
    nova_populacija = populacija
    stevilo_parjenj = poisson(t = len(populacija))
    for k in range(stevilo_parjenj):
        starsa = random.sample(populacija, k = 2)
        nova_populacija.append(crossover(n, starsa[0], starsa[1]))
    return nova_populacija


In [74]:
def KoncniTestGA(konec,maksimalna):
    """
    Konec pove, koliko generacij naj pogledamo.
    Funkcija lahko vrne dve možnosti:
                                     1. Najde nam izjemo, jo pokaže in tako ovržemo domnevo.
                                     2. Izjeme sploh ne dobimo, in tako izjeme ne vrnemo.
    """
    populacija = testna_populacija(maksimalna)
    i = 1
    while i <= konec:
        populacija = potomci(populacija)
        populacija = mutacija(populacija)
        populacija = testna_populacija(populacija)
        for i in range(len(populacija)):
            G = populacija[i]
            if neodvistnostno_stevilo(G) <= 1 + povprecna(G):
                if G.hamiltonian_path() == None:
                    print (G)
                    return "Ovrgli smo domnevo"

                i += 1
    return "Domneva ni ovrzena."

In [92]:
f = KoncniTestGA(1,20)

KeyboardInterrupt: 

In [93]:
f = KoncniTestGA(1,5)

In [0]:
f